In [1]:
import pandas as pd
import numpy as np
from surprise import KNNBasic,KNNWithMeans
from surprise import accuracy
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from catboost import CatBoostClassifier
import pandas as pd

In [8]:
visit_data = pd.read_parquet('visit_data.parquet')

In [9]:
visit_data

,user_id,age,gender,name,address,pos1,pos2,user_rating
0,1,18,Мужской,Fox,"Парк Фили, Большая Филёвская улица, 22 ст16а, ...",37.490661,55.750929,5
1,1,18,Мужской,Кремлёвский,"1-я Тверская-Ямская улица, 3 ст1, Москва",37.594379,55.770681,2
2,1,18,Мужской,Soul Burger,"Большой Сухаревский переулок, 25 ст1, Москва",37.632115,55.770884,5
3,2,22,Мужской,МУ-МУ,"Казанский вокзал, Комсомольская площадь, 2, Мо...",37.656479,55.773491,3
4,2,22,Мужской,Москва-Дели,"Ермолаевский переулок, 7, Москва",37.590505,55.764352,5
...,...,...,...,...,...,...,...,...
569467,99999,48,Женский,Центр театра и кино под руководством Никиты Ми...,"Улица Поварская, 33 ст1, Москва",37.586203,55.756832,4
569468,99999,48,Женский,Московский Драматический театр им. М.Н. Ермоловой,"Тверская улица, 5/6, Москва",37.612017,55.757425,5
569469,100000,47,Женский,Кашемир,"ТКЗ Дворец на Яузе, площадь Журавлева, 1, Москва",37.704673,55.784040,5
569470,100000,47,Женский,Мосспектакль,"Улица Покровка, 40Б, Москва",37.652562,55.761082,5


### Добавление искусственной информации о том посетил ли пользователь место или посмотрел подробнее

In [11]:
visit_data['visited'] = np.random.choice([0, 1], size=len(visit_data), p=[0.8, 0.2])
# Просмотр подробной информации (40% вероятность)
visit_data['viewed_detail'] = np.random.choice([0, 1], size=len(visit_data), p=[0.6, 0.4])

In [12]:
visit_data

,user_id,age,gender,name,address,pos1,pos2,user_rating,visited,viewed_detail
0,1,18,Мужской,Fox,"Парк Фили, Большая Филёвская улица, 22 ст16а, ...",37.490661,55.750929,5,1,1
1,1,18,Мужской,Кремлёвский,"1-я Тверская-Ямская улица, 3 ст1, Москва",37.594379,55.770681,2,0,1
2,1,18,Мужской,Soul Burger,"Большой Сухаревский переулок, 25 ст1, Москва",37.632115,55.770884,5,0,0
3,2,22,Мужской,МУ-МУ,"Казанский вокзал, Комсомольская площадь, 2, Мо...",37.656479,55.773491,3,1,0
4,2,22,Мужской,Москва-Дели,"Ермолаевский переулок, 7, Москва",37.590505,55.764352,5,0,1
...,...,...,...,...,...,...,...,...,...,...
569467,99999,48,Женский,Центр театра и кино под руководством Никиты Ми...,"Улица Поварская, 33 ст1, Москва",37.586203,55.756832,4,0,0
569468,99999,48,Женский,Московский Драматический театр им. М.Н. Ермоловой,"Тверская улица, 5/6, Москва",37.612017,55.757425,5,0,0
569469,100000,47,Женский,Кашемир,"ТКЗ Дворец на Яузе, площадь Журавлева, 1, Москва",37.704673,55.784040,5,0,0
569470,100000,47,Женский,Мосспектакль,"Улица Покровка, 40Б, Москва",37.652562,55.761082,5,0,0


### Коллаборативная фильтрация

In [6]:
data = visit_data.copy()

In [11]:
# Подготовка данных для surprise
reader = Reader(rating_scale=(1, 5))  # Рейтинги в вашем датасете от 1 до 5
data = Dataset.load_from_df(data[['user_id', 'name', 'user_rating']], reader)

# Разделение на обучающую и тестовую выборки
trainset, testset = train_test_split(data, test_size=0.25)

### Модель KNN

In [12]:
# Создание и обучение модели
model = KNNBasic()
model.fit(trainset)

# Предсказания на тестовом наборе
predictions = model.test(testset)

# Расчет RMSE (Root Mean Squared Error)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.4464


1.4463887995045754

In [13]:
# Предсказания на тестовом наборе
predictions = model.test(testset)

In [11]:
from collections import defaultdict
from surprise import accuracy

def precision_recall_at_k(predictions, k=10, threshold=4):
    '''Возвращает словарь с precision и recall по пользователю для заданного k и порога.'''

    # Словарь для хранения результатов
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()

    for uid, user_ratings in user_est_true.items():
        # Сортируем предсказания пользователя по убыванию предсказанного рейтинга
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Число рекомендованных объектов, рейтинг которых выше порога
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings[:k])

        # Число релевантных и рекомендованных объектов
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Число релевантных объектов
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

        # Precision@K: Процент рекомендованных объектов, которые релевантны
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Процент релевантных объектов, которые были рекомендованы
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [14]:
# Расчет метрик
precisions, recalls = precision_recall_at_k(predictions, k=1, threshold=4)

# Средний precision и recall по всем пользователям
print(f"Precision@5: {sum(prec for prec in precisions.values()) / len(precisions):.4f}")
print(f"Recall@5: {sum(rec for rec in recalls.values()) / len(recalls):.4f}")

Precision@5: 0.8359
Recall@5: 0.6417


In [15]:
predictions

[Prediction(uid=2213, iid='Osteria Unica', r_ui=2.0, est=3.4208807161867925, details={'actual_k': 26, 'was_impossible': False}),
 Prediction(uid=39406, iid='Пестики-тычинки', r_ui=5.0, est=4.13684210526316, details={'actual_k': 18, 'was_impossible': False}),
 Prediction(uid=98225, iid='Театр на Таганке', r_ui=5.0, est=3.741077931536781, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=84638, iid='Библиотека №225 им. М.Н. Задорнова', r_ui=5.0, est=3.3616858237547893, details={'actual_k': 13, 'was_impossible': False}),
 Prediction(uid=23124, iid='RedBox', r_ui=1.0, est=3.6280814576634546, details={'actual_k': 26, 'was_impossible': False}),
 Prediction(uid=86319, iid='Библиотека №101', r_ui=2.0, est=4.0600000000000005, details={'actual_k': 13, 'was_impossible': False}),
 Prediction(uid=24463, iid='Москино Вымпел', r_ui=4.0, est=3.698422398291751, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid=66613, iid='Eshak', r_ui=5.0, est=3.431

### Инференс модели

In [ ]:
data = visit_data.copy()
# Подготовка данных для surprise
reader = Reader(rating_scale=(1, 5))  # Рейтинги в вашем датасете от 1 до 5
data = Dataset.load_from_df(data[['user_id', 'name', 'user_rating']], reader)
trainset = data.build_full_trainset()
sim_options = {
    'name': 'pearson_baseline',  # Метрика схожести: косинусное сходство, MSD, pearson, pearson_baseline
    'user_based': True,  # или False, если предпочитаете схожесть на основе объектов
    'min_support': 5,  # Минимальное количество общих объектов/пользователей
    'shrinkage': 0  # Параметр уменьшения для pearson_baseline, 100 по умолчанию
}

# Создание и обучение модели KNNWithMeans
inference_model = KNNWithMeans(k=40, sim_options=sim_options)
cross_validate(inference_model, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
inference_model.fit(trainset)

Estimating biases using als...


In [1]:
1

1

In [20]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
# Допустим, мы хотим сгенерировать рекомендации для пользователя с user_id
# Обучение модели
users = trainset.all_users()
items = trainset.all_items()
item_names = trainset.to_raw_iid  # Получаем исходные идентификаторы объектов

# Предположим, мы хотим сгенерировать рекомендации для пользователя с исходным user_id
user_id = 16  # Пример исходного ID пользователя
inner_user_id = trainset.to_inner_uid(user_id)  # Преобразуем во внутренний ID

# Получаем список объектов, которые пользователь уже оценил
user_rated_items = [item for (item, _) in trainset.ur[inner_user_id]]

# Генерация предсказаний для объектов, которые пользователь не оценил
unrated_items = [item for item in items if item not in user_rated_items]
predictions = [inference_model.predict(inner_user_id, item) for item in unrated_items]

# Сортировка предсказаний по убыванию предсказанного рейтинга
predictions.sort(key=lambda x: x.est, reverse=True)

# Выбор top-N рекомендаций
N = 100
top_n_recommendations = predictions[:N]

# Вывод рекомендаций
print(f"Top-{N} Recommendations for User {user_id}:")
for pred in top_n_recommendations:
    print(f"Item ID: {item_names(pred.iid)}, Predicted Rating: {pred.est}")

Top-100 Recommendations for User 16:
Item ID: Fox, Predicted Rating: 3.6990914390874354
Item ID: Кремлёвский, Predicted Rating: 3.6990914390874354
Item ID: Soul Burger, Predicted Rating: 3.6990914390874354
Item ID: МУ-МУ, Predicted Rating: 3.6990914390874354
Item ID: Москва-Дели, Predicted Rating: 3.6990914390874354
Item ID: Краснодарский Парень, Predicted Rating: 3.6990914390874354
Item ID: PARKHOUSE, Predicted Rating: 3.6990914390874354
Item ID: Vasilchuki chaihona №1, Predicted Rating: 3.6990914390874354
Item ID: Ботаника, Predicted Rating: 3.6990914390874354
Item ID: Nagoya, Predicted Rating: 3.6990914390874354
Item ID: Amico Bistro, Predicted Rating: 3.6990914390874354
Item ID: Metafora bar, Predicted Rating: 3.6990914390874354
Item ID: Этажерка, Predicted Rating: 3.6990914390874354
Item ID: Кот Ч, Predicted Rating: 3.6990914390874354
Item ID: Liberty, Predicted Rating: 3.6990914390874354
Item ID: Faiza, Predicted Rating: 3.6990914390874354
Item ID: Dry&wet, Predicted Rating: 3.69

In [18]:
visit_data[visit_data['user_id']==16]

,user_id,age,gender,name,address,pos1,pos2,user_rating,visited,viewed_detail
94,16,19,Мужской,Нах,"Тверская улица, 9, Москва",37.611009,55.759413,5,0,1
95,16,19,Мужской,Петрович,"Улица Мясницкая, 24 ст3, Москва",37.635206,55.762667,5,0,0
96,16,19,Мужской,Весна,"ТЦ Московский, Московский бульвар, 1а, Балаших...",37.960486,55.825886,5,0,0
97,16,19,Мужской,Танzпол,"Шараповская улица, 7, Мытищи, Московская область",37.751242,55.915195,4,0,0
98,16,19,Мужской,Geisha Superman,"Улица Малая Бронная, 2 ст1, Москва",37.598300,55.759026,3,0,1
99,16,19,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,5,0,0
100,16,19,Мужской,The Бык,"ТДЦ ВЭБ.РФ, Новинский бульвар, 31, Москва",37.582520,55.757830,4,0,0
101,16,19,Мужской,Twenty One Club,"БЦ Обновление Арбата, улица Новый Арбат, 21, М...",37.586733,55.751924,5,0,0


In [19]:
visit_data[visit_data['name']=='Happy']

,user_id,age,gender,name,address,pos1,pos2,user_rating,visited,viewed_detail
99,16,19,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,5,0,0
898,137,24,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,2,0,0
1332,203,23,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,1,0,0
1471,221,20,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,5,0,1
2164,329,18,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,5,0,0
...,...,...,...,...,...,...,...,...,...,...
78816,12107,21,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,5,0,0
79825,12255,19,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,4,0,0
80199,12309,18,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,4,0,0
80246,12316,21,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,1,0,1


In [ ]:
predictions

### Модель SVD

In [ ]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate
data = visit_data.copy()
# Подготовка данных для surprise
reader = Reader(rating_scale=(1, 5))  # Рейтинги в вашем датасете от 1 до 5
data = Dataset.load_from_df(data[['user_id', 'name', 'user_rating']], reader)

# Разделение на обучающую и тестовую выборки
trainset, testset = train_test_split(data, test_size=0.25)

# Использование SVD
algo = SVD()

# Проведение кросс-валидации для оценки модели
cross_validate(algo, trainset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [21]:
from surprise.model_selection import GridSearchCV

param_grid = {
    'n_epochs': [5, 10, 20],  # Количество эпох обучения
    'lr_all': [0.002, 0.005],  # Скорость обучения
    'reg_all': [0.4, 0.6]  # Коэффициент регуляризации
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Лучшие результаты RMSE
print(gs.best_score['rmse'])

# Параметры для лучшего RMSE
print(gs.best_params['rmse'])

1.3463557852571846
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}


In [22]:
# Обучение модели с лучшими параметрами
algo = SVD(n_epochs=gs.best_params['rmse']['n_epochs'],
           lr_all=gs.best_params['rmse']['lr_all'],
           reg_all=gs.best_params['rmse']['reg_all'])
algo.fit(trainset)

In [24]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
# Допустим, мы хотим сгенерировать рекомендации для пользователя с user_id = 1
# Обучение модели
users = trainset.all_users()
items = trainset.all_items()
item_names = trainset.to_raw_iid  # Получаем исходные идентификаторы объектов

# Предположим, мы хотим сгенерировать рекомендации для пользователя с исходным user_id
user_id = 16  # Пример исходного ID пользователя
inner_user_id = trainset.to_inner_uid(user_id)  # Преобразуем во внутренний ID

# Получаем список объектов, которые пользователь уже оценил
user_rated_items = [item for (item, _) in trainset.ur[inner_user_id]]

# Генерация предсказаний для объектов, которые пользователь не оценил
unrated_items = [item for item in items if item not in user_rated_items]
predictions = [inference_model.predict(inner_user_id, item) for item in unrated_items]

# Сортировка предсказаний по убыванию предсказанного рейтинга
predictions.sort(key=lambda x: x.est, reverse=True)

# Выбор top-N рекомендаций
N = 100
top_n_recommendations = predictions[:N]

# Вывод рекомендаций
print(f"Top-{N} Recommendations for User {user_id}:")
for pred in top_n_recommendations:
    print(f"Item ID: {item_names(pred.iid)}, Predicted Rating: {pred.est}")

Top-100 Recommendations for User 16:
Item ID: Eno bistro, Predicted Rating: 3.6990914390874354
Item ID: Up Sushi, Predicted Rating: 3.6990914390874354
Item ID: Библиотека №170 им. К.Г. Паустовского, Predicted Rating: 3.6990914390874354
Item ID: Savva, Predicted Rating: 3.6990914390874354
Item ID: Библиотека №51, Predicted Rating: 3.6990914390874354
Item ID: Шоколадница, Predicted Rating: 3.6990914390874354
Item ID: Sartoria Lamberti, Predicted Rating: 3.6990914390874354
Item ID: Osteria Gini, Predicted Rating: 3.6990914390874354
Item ID: The Abbey Players Pub, Predicted Rating: 3.6990914390874354
Item ID: Mission, Predicted Rating: 3.6990914390874354
Item ID: Московский открытый студенческий театр, Predicted Rating: 3.6990914390874354
Item ID: Московский Государственный Академический театр оперетты, Predicted Rating: 3.6990914390874354
Item ID: Бочка, Predicted Rating: 3.6990914390874354
Item ID: Московский Драматический театр им. М.Н. Ермоловой, Predicted Rating: 3.6990914390874354
It

In [29]:
# Предсказания на тестовом наборе
predictions = algo.test(testset)

In [30]:
# Расчет метрик
precisions, recalls = precision_recall_at_k(predictions, k=1, threshold=4)

# Средний precision и recall по всем пользователям
print(f"Precision@5: {sum(prec for prec in precisions.values()) / len(precisions):.4f}")
print(f"Recall@5: {sum(rec for rec in recalls.values()) / len(recalls):.4f}")

Precision@5: 0.9999
Recall@5: 0.4023


### Гибридная рекомендательная система

In [16]:
from sklearn.model_selection import train_test_split
df = visit_data.copy()
df['name'] = df['name'].astype('category')
df['address'] = df['address'].astype('category')

# Создаем новую колонку 'place_id'
df['place_id'] = df['name'].astype(str) + ' - ' + df['address'].astype(str)
df['place_id'] = df['place_id'].astype('category')

# Преобразование рейтингов и пола пользователя
df['rating_class'] = (df['user_rating'] >= 4).astype(int)
df['gender'] = df['gender'].astype('category')

# Подготовка признаков и целевой переменной
X = df[['user_id', 'age', 'gender', 'place_id']]  # Исключаем ненужные колонки
y = df['rating_class']

# CatBoost умеет работать с категориальными переменными напрямую
cat_features = ['gender', 'place_id']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Инициализация и обучение модели CatBoostClassifier
classifier = CatBoostClassifier(iterations=500,
                                learning_rate=0.05,
                                depth=4,
                                verbose=50,
                                thread_count=-1)
classifier.fit(X_train, y_train, cat_features=cat_features)

0:	learn: 0.6912048	total: 203ms	remaining: 1m 41s
50:	learn: 0.6731845	total: 3.43s	remaining: 30.2s
100:	learn: 0.6730446	total: 7s	remaining: 27.7s
150:	learn: 0.6730070	total: 9.64s	remaining: 22.3s
200:	learn: 0.6729733	total: 11.6s	remaining: 17.3s
250:	learn: 0.6729341	total: 13.5s	remaining: 13.4s
300:	learn: 0.6728915	total: 15.5s	remaining: 10.2s
350:	learn: 0.6728334	total: 17.5s	remaining: 7.44s
400:	learn: 0.6727871	total: 19.6s	remaining: 4.83s
450:	learn: 0.6727398	total: 21.6s	remaining: 2.35s
499:	learn: 0.6726952	total: 23.6s	remaining: 0us


In [17]:
y_test

455577    0
455578    1
455579    0
455580    1
455581    0
         ..
569467    1
569468    1
569469    1
569470    1
569471    1
Name: rating_class, Length: 113895, dtype: int64

In [18]:
from sklearn.metrics import precision_recall_curve, average_precision_score
import numpy as np

# Предсказание вероятностей для тестовой выборки
y_scores = classifier.predict_proba(X_test)[:, 1]

# Расчет precision и recall для всех возможных порогов
precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

# Расчет Average Precision (AP)
ap_score = average_precision_score(y_test, y_scores)


print(f'Average Precision (AP): {ap_score:.4f}')
k = 15
# Опционально: Найти precision и recall при конкретном k, используя пороги
# Например, для порога, соответствующего top-k предсказаний
threshold_k = 0.55
precision_at_k = precision[np.searchsorted(thresholds, threshold_k)]
recall_at_k = recall[np.searchsorted(thresholds, threshold_k)]

print(f'Precision@{k}: {precision_at_k:.4f}, Recall@{k}: {recall_at_k:.4f}')

Average Precision (AP): 0.6007
Precision@15: 0.6000, Recall@15: 0.9333


In [19]:
def prepare_inference_data(user_id, user_features, all_places):
    """
    Создает DataFrame для инференса для одного пользователя с учетом всех неоцененных мест.
    user_id: ID пользователя, для которого генерируются рекомендации
    user_features: Демографические данные пользователя (например, возраст и пол)
    all_places: DataFrame всех возможных мест с их информацией (включая названия и адреса)
    Возвращает DataFrame, подходящий для модели предсказания.
    """
    # Создаем список словарей, где каждый словарь соответствует одной строке в будущем DataFrame
    data_rows = [{
        'user_id': user_id,
        'age': user_features['age'],
        'gender': user_features['gender'],
        'place_id': row['place_id']  # Используем place_id напрямую из all_places
    } for index, row in all_places.iterrows()]

    # Преобразуем список словарей в DataFrame
    data = pd.DataFrame(data_rows)

    return data

# Убедитесь, что all_places содержит уникальные идентификаторы мест (place_id), включая название и адрес
all_places = df[['place_id']].drop_duplicates()

# Подготовка данных для инференса
user_id = 16
user_features = {'age': 25, 'gender': 'M'}
X_infer = prepare_inference_data(user_id, user_features, all_places)

# Теперь X_infer должен содержать корректную структуру для инференса: 
# одна строка для каждой пары "пользователь-место", где место еще не оценено данным пользователем.

In [20]:
# Предполагается, что X_infer - это DataFrame, подготовленный для инференса,
# и predictions - массив вероятностей положительного класса для каждого места
predictions = classifier.predict_proba(X_infer)[:, 1]
# Добавляем вероятности как новый столбец в X_infer
X_infer['probability'] = predictions

# Сортируем места по убыванию вероятности
X_infer_sorted = X_infer.sort_values(by='probability', ascending=False)

# Выбираем топ-N мест
N = 100
top_n_recommendations = X_infer_sorted.head(N)

# Выводим топ-N рекомендаций
print("Топ-{} рекомендаций для пользователя с ID {}:".format(N, user_id))
print(top_n_recommendations[['place_id', 'probability']])

Топ-100 рекомендаций для пользователя с ID 16:
                                               place_id  probability
304                Ra`men - Пятницкая улица, 45, Москва     0.633103
17      Beverly hills diner - Улица Сретенка, 1, Москва     0.632579
281   Petruccio. Pizza&pasta - РЦ Высота, улица Юных...     0.629992
477          Молодежь - Сущёвская улица, 21 ст8, Москва     0.629195
231   Live stars - Красный Октябрь, Берсеневский пер...     0.629062
...                                                 ...          ...
1222  Орфей - ДК Балашиха, Московский проезд, 9, Бал...     0.613100
277          Semenov - Китайгородский проезд, 5, Москва     0.612966
276   Руки Вверх! - Московский проспект, 56, Пушкино...     0.612962
148   Atlas Club - ДЛ Рассвет, Столярный переулок, 3...     0.612962
185   Вираж - Центральная улица, 98, д. Осеево, Лоси...     0.612930

[100 rows x 2 columns]


In [76]:
visit_data[visit_data['user_id']==16]

,user_id,age,gender,name,address,pos1,pos2,user_rating
94,16,19,Мужской,Нах,"Тверская улица, 9, Москва",37.611009,55.759413,5
95,16,19,Мужской,Петрович,"Улица Мясницкая, 24 ст3, Москва",37.635206,55.762667,5
96,16,19,Мужской,Весна,"ТЦ Московский, Московский бульвар, 1а, Балаших...",37.960486,55.825886,5
97,16,19,Мужской,Танzпол,"Шараповская улица, 7, Мытищи, Московская область",37.751242,55.915195,4
98,16,19,Мужской,Geisha Superman,"Улица Малая Бронная, 2 ст1, Москва",37.598300,55.759026,3
99,16,19,Мужской,Happy,"Ансамбль Миусского трамвайного депо, улица Лес...",37.593084,55.780218,5
100,16,19,Мужской,The Бык,"ТДЦ ВЭБ.РФ, Новинский бульвар, 31, Москва",37.582520,55.757830,4
101,16,19,Мужской,Twenty One Club,"БЦ Обновление Арбата, улица Новый Арбат, 21, М...",37.586733,55.751924,5


In [ ]:
classifier

In [21]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(classifier, f)

In [23]:
places_data = pd.read_parquet('places_data.parquet')

In [25]:
category_subcategory_data = places_data[['subcategory','category']].drop_duplicates()

In [26]:
category_subcategory_data.to_csv('sub_cat_and_cat_data.csv')

In [27]:
category_subcategory_data

,subcategory,category
0,Кафе,Поесть
500,Рестораны,Поесть
966,Бары,Поесть
1440,Суши-бары,Поесть
1940,Быстрое питание,Поесть
...,...,...
54592,Студии медитации,Спорт
29071,Продажа живых бабочек,Спецмагазины
669,Акватеррариумы,Развлечения
7809,"Букмекеры, лотереи",Развлечения


In [28]:
df = category_subcategory_data.copy()
# Генерируем уникальные ID для категорий
category_ids = pd.factorize(df['category'])[0] + 1
df['category_id'] = category_ids

# Генерируем уникальные ID для подкатегорий
subcategory_combinations = df['category'] + df['subcategory']
subcategory_ids = pd.factorize(subcategory_combinations)[0] + 1
df['subcategory_id'] = subcategory_ids

# Создаем отдельные DataFrame'ы для категорий и подкатегорий
categories_df = df[['category_id', 'category']].drop_duplicates().reset_index(drop=True)
subcategories_df = df[['subcategory_id', 'category_id', 'subcategory']].drop_duplicates().reset_index(drop=True)

print(categories_df)
print(subcategories_df)

   category_id      category
0            1        Поесть
1            2   Развлечения
2            3        Туризм
3            4        Товары
4            5        Услуги
5            6  Спецмагазины
6            7      Продукты
7            8         Спорт
8            9   Образование
     subcategory_id  category_id            subcategory
0                 1            1                   Кафе
1                 2            1              Рестораны
2                 3            1                   Бары
3                 4            1              Суши-бары
4                 5            1        Быстрое питание
..              ...          ...                    ...
237             238            8       Студии медитации
238             239            6  Продажа живых бабочек
239             240            2         Акватеррариумы
240             241            2     Букмекеры, лотереи
241             242            2               Мавзолей

[242 rows x 3 columns]


In [38]:
categories_df.to_csv('categories.csv',index=False)

In [39]:
subcategories_df.to_csv('subcategories.csv',index=False)

In [35]:
categories_df

,category_id,category
0,1,Поесть
1,2,Развлечения
2,3,Туризм
3,4,Товары
4,5,Услуги
5,6,Спецмагазины
6,7,Продукты
7,8,Спорт
8,9,Образование


In [37]:
subcategories_df.rename(columns={'subcategory_id':'id','subcategory':'name'},inplace=True)
categories_df.rename(columns={'category_id':'id','category':'name'},inplace=True)

In [58]:
NEW_PLACES_DATA = pd.read_csv('../COURSE_WRK/some_marked_places.csv',header=None)

/var/folders/5m/skwg1kcj24198_7b50yg8twm0000gp/T/ipykernel_43155/3937785881.py:1: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  NEW_PLACES_DATA = pd.read_csv('../COURSE_WRK/some_marked_places.csv',header=None)


In [60]:
NEW_PLACES_DATA = NEW_PLACES_DATA.drop(0,axis=1)

In [61]:
NEW_PLACES_DATA

,1,2,3,4,5,6,7,8,9,10
0,Поесть,Кафе,МУ-МУ,"Охотный ряд, Манежная площадь, 1 ст2, Москва",4.6,894 оценки,37.615036,55.755489,"Кафе «Му-му» — это место, где можно по-домашне...",https://ams2-cdn.2gis.com/previews/10286469513...
1,Поесть,Кафе,Кафе на Останкинской башне,"Улица Академика Королёва, 15 к1, Москва",4.7,62 оценки,37.611677,55.819868,Останкинская телебашня предоставляет большой с...,https://i1.photo.2gis.com/images/branch/0/3025...
2,Поесть,Кафе,Здрасте,"Лефортово Парк, Красноказарменная улица, 14а к...",4.9,21 оценка,37.706236,55.752525,«Здрасте» — сеть уютных кофеен с авторской кух...,https://ams2-cdn.2gis.com/previews/11437209486...
3,Поесть,Кафе,Краснодар,"Улица Большая Дмитровка, 9 ст1, Москва",4.7,140 оценок,37.614207,55.761462,ООО «МИЛОВИДОВ».,https://ams2-cdn.2gis.com/previews/10365511406...
4,Поесть,Кафе,KIDZROCK Cafe,"ТРЦ Хорошо!, Хорошёвское шоссе, 27, Москва",4.9,50 оценок,37.523658,55.777112,«Замания» — это более 2000 квадратных метров н...,https://ams2-cdn.2gis.com/previews/11351735832...
...,...,...,...,...,...,...,...,...,...,...
67448,Спорт,Яхт-клубы,Поместье галс,"Прибрежная улица, ст2в/1, д. Болтино, Мытищи г...",Нет данных,Нет данных,37.681227,55.973300,NaN,NaN
67449,Спорт,Яхт-клубы,Сенеж,Солнечногорск,Нет данных,Нет данных,36.977631,56.185107,NaN,NaN
67450,Спорт,Яхт-клубы,Фарватер-вилла,"Прибрежная улица, 5, д. Подрезово, Мытищи горо...",5,1 оценка,37.698313,55.972957,NaN,NaN
67451,Спорт,Яхт-клубы,Химик,"Набережная улица, 6, Долгопрудный, Московская ...",Нет данных,Нет данных,37.512856,55.955171,NaN,NaN


In [62]:
columns_names = ['category','subcategory','name','address','rating','reviews_count','pos1','pos2','description','image']
NEW_PLACES_DATA.columns = columns_names

In [63]:
NEW_PLACES_DATA['description'] = NEW_PLACES_DATA['description'].fillna('Нет данных')
NEW_PLACES_DATA['image'] = NEW_PLACES_DATA['image'].fillna('Нет данных')

In [66]:
NEW_PLACES_DATA[(NEW_PLACES_DATA['description']!='Нет данных')&(NEW_PLACES_DATA['image']!='Нет данных')]

,category,subcategory,name,address,rating,reviews_count,pos1,pos2,description,image
0,Поесть,Кафе,МУ-МУ,"Охотный ряд, Манежная площадь, 1 ст2, Москва",4.6,894 оценки,37.615036,55.755489,"Кафе «Му-му» — это место, где можно по-домашне...",https://ams2-cdn.2gis.com/previews/10286469513...
1,Поесть,Кафе,Кафе на Останкинской башне,"Улица Академика Королёва, 15 к1, Москва",4.7,62 оценки,37.611677,55.819868,Останкинская телебашня предоставляет большой с...,https://i1.photo.2gis.com/images/branch/0/3025...
2,Поесть,Кафе,Здрасте,"Лефортово Парк, Красноказарменная улица, 14а к...",4.9,21 оценка,37.706236,55.752525,«Здрасте» — сеть уютных кофеен с авторской кух...,https://ams2-cdn.2gis.com/previews/11437209486...
3,Поесть,Кафе,Краснодар,"Улица Большая Дмитровка, 9 ст1, Москва",4.7,140 оценок,37.614207,55.761462,ООО «МИЛОВИДОВ».,https://ams2-cdn.2gis.com/previews/10365511406...
4,Поесть,Кафе,KIDZROCK Cafe,"ТРЦ Хорошо!, Хорошёвское шоссе, 27, Москва",4.9,50 оценок,37.523658,55.777112,«Замания» — это более 2000 квадратных метров н...,https://ams2-cdn.2gis.com/previews/11351735832...
...,...,...,...,...,...,...,...,...,...,...
61003,Спорт,Аренда спортивных площадок,Evolve gym,"Новоспасский, Дербеневская набережная, 7 ст6, ...",4.9,11 оценок,37.653936,55.721914,"Хочется чего-нибудь сытного, вкусного и по при...",https://i4.photo.2gis.com/images/branch/0/3025...
61004,Спорт,Аренда спортивных площадок,Fit N Go,"Улица 3-е Почтовое отделение, 102, Люберцы, Мо...",4.3,6 оценок,37.865220,55.681600,"ООО «Меркато», Москва, ОГРН 1187746394296.",https://ams2-cdn.2gis.com/previews/10778388854...
61005,Спорт,Аренда спортивных площадок,Fit4ever-fitness&ems,"Рождественский бульвар, 9 ст1, Москва",5,7 оценок,37.625756,55.766876,"Хочется чего-нибудь сытного, вкусного и по при...",https://i9.photo.2gis.com/images/branch/0/3025...
61011,Спорт,Аренда спортивных площадок,TopStar,"Улица Декабристов, 27, Москва",4.3,9 оценок,37.609488,55.863151,"ООО «Додо Франчайзинг», Сыктывкар, ОГРН 113110...",https://ams2-cdn.2gis.com/previews/10778388854...


In [67]:
NEW_PLACES_DATA.to_parquet('new_places_data.parquet')

In [68]:
NEW_PLACES_DATA.to_csv('new_places_data.csv',index=False)

In [69]:
NEW_PLACES_DATA

,category,subcategory,name,address,rating,reviews_count,pos1,pos2,description,image
0,Поесть,Кафе,МУ-МУ,"Охотный ряд, Манежная площадь, 1 ст2, Москва",4.6,894 оценки,37.615036,55.755489,"Кафе «Му-му» — это место, где можно по-домашне...",https://ams2-cdn.2gis.com/previews/10286469513...
1,Поесть,Кафе,Кафе на Останкинской башне,"Улица Академика Королёва, 15 к1, Москва",4.7,62 оценки,37.611677,55.819868,Останкинская телебашня предоставляет большой с...,https://i1.photo.2gis.com/images/branch/0/3025...
2,Поесть,Кафе,Здрасте,"Лефортово Парк, Красноказарменная улица, 14а к...",4.9,21 оценка,37.706236,55.752525,«Здрасте» — сеть уютных кофеен с авторской кух...,https://ams2-cdn.2gis.com/previews/11437209486...
3,Поесть,Кафе,Краснодар,"Улица Большая Дмитровка, 9 ст1, Москва",4.7,140 оценок,37.614207,55.761462,ООО «МИЛОВИДОВ».,https://ams2-cdn.2gis.com/previews/10365511406...
4,Поесть,Кафе,KIDZROCK Cafe,"ТРЦ Хорошо!, Хорошёвское шоссе, 27, Москва",4.9,50 оценок,37.523658,55.777112,«Замания» — это более 2000 квадратных метров н...,https://ams2-cdn.2gis.com/previews/11351735832...
...,...,...,...,...,...,...,...,...,...,...
67448,Спорт,Яхт-клубы,Поместье галс,"Прибрежная улица, ст2в/1, д. Болтино, Мытищи г...",Нет данных,Нет данных,37.681227,55.973300,Нет данных,Нет данных
67449,Спорт,Яхт-клубы,Сенеж,Солнечногорск,Нет данных,Нет данных,36.977631,56.185107,Нет данных,Нет данных
67450,Спорт,Яхт-клубы,Фарватер-вилла,"Прибрежная улица, 5, д. Подрезово, Мытищи горо...",5,1 оценка,37.698313,55.972957,Нет данных,Нет данных
67451,Спорт,Яхт-клубы,Химик,"Набережная улица, 6, Долгопрудный, Московская ...",Нет данных,Нет данных,37.512856,55.955171,Нет данных,Нет данных
